In [89]:
import numpy as np
import scipy.stats as spstats
from scipy import signal
import pickle
from multiprocessing import Pool
import multiprocessing
import scipy.sparse as sparse
import seaborn as sns

from potentials import IndependentPotential
from baselines import GenerateSigma,construct_ESVM_kernel,set_function
from optimize import Run_eval_test,optimize_parallel_new 
from samplers import MCMC_sampler,Generate_train
from utils import *

In [90]:
N_train = 1*10**4 # Number of samples on which we optimize
N_test = 1*10**4 # Number of samples
n_traj_train = 1
n_traj_test = 100 # Number of independent MCMC trajectories for test
f_type = "cos_sum"

In [91]:
sampler = {"sampler":"3rd_poly","burn_type":"full","main_type":"full"}# Sampling method

In [92]:
d = 10
typ = sampler["sampler"]
#mu = np.array([0.0],dtype = float)
#Sigma = 1.0
mu = np.zeros(d, dtype = float)
Sigma = np.eye(d)
#Sigma = GenerateSigma(d,rand_seed = 777,eps = 0.1) #covariation matrix 
"""
params_distr = {
    "mu":mu,
    "Sigma":Sigma,
    "d":d
}

params_distr = {
    "mu":0.0,
    "lambda":1.0,
    "d":d
}
"""
params_distr = {
    "b":2.0,
    "d":d
}

Cur_pot = IndependentPotential(typ,params_distr)
#Cur_pot = GausMixtureSame(Sigma,mu,p)
#Cur_pot = GausMixtureIdent(mu,p)

In [93]:
rand_seed = 777
traj,traj_grad = Cur_pot.sample(rand_seed,N_train)
traj = np.expand_dims(traj,axis=0)
traj_grad = np.expand_dims(traj_grad,axis=0)
print(traj.shape)
print(traj_grad.shape)

(1, 10000, 10)
(1, 10000, 10)


In [94]:
inds_arr = np.array([0]) # Taking the second index (not intercept)
params = None 
f_vals = set_function(f_type,traj,inds_arr,params)

In [95]:
print(f_vals.shape)

(1, 10000, 1)


In [96]:
W_train_spec = None
W_test_spec = None
degree = 5 #degree of the polynomial
opt_structure_train = {
    "W":W_train_spec,
    "n_restarts": 5, # Number of restarts during optimization,
    "sigma": 3.0, # Deviation of starting points
    "tol": 1e-6, # Tolerance (for the norm of gradient)
    "alpha": 0.0, # Ridge penalty for 2nd order control functionals
    "beta": 1e2 # smoothing parameter in the softmax
}
methods = ["EVM","LS","MAX"]

In [97]:
coef_dict_k_deg = optimize_parallel_new(degree,inds_arr,f_vals,traj,traj_grad,opt_structure_train,methods)

k-th degree optimization terminated succesfully
Jacobian matrix at termination: 
[ 1.71459698e-10  2.43657735e-09 -2.23506010e-09  2.31665569e-09
  8.96424598e-10 -5.14625758e-09 -8.65826184e-09 -2.32439933e-09
 -9.10728154e-10 -6.00567622e-11  2.55371533e-09  4.55307806e-09
 -4.90586635e-09  5.30055148e-09  3.63639105e-10 -8.83930919e-10
  2.09723325e-09 -1.26219699e-08 -6.93226228e-09  3.53154189e-09
  3.22145627e-10  6.66788768e-10  4.13366240e-10 -1.38101959e-10
 -3.18732375e-10 -1.98319067e-10  2.13742331e-09 -6.65110679e-10
  5.60695916e-11 -4.59746287e-10 -1.51606754e-08  9.58026877e-09
 -4.73691870e-09 -2.40673425e-09  1.15023208e-08 -1.45196537e-09
 -4.06376430e-09 -5.35187680e-09 -1.38826668e-08 -1.12175640e-08
 -4.01361692e-08 -1.27811324e-06 -3.43981027e-08  8.85862807e-10
  2.16659816e-08  1.62252610e-05 -1.31138269e-07 -1.90674722e-07
  7.34877969e-08 -2.58486343e-07]
k-th degree optimization terminated succesfully
Jacobian matrix at termination: 
[-6.24915603e-10 -8.0993

### Coefficients for given training methods

In [98]:
print(coef_dict_k_deg["EVM"])

[[-9.04841569e-04 -2.16898572e-03 -1.90877232e-04  1.49131716e-03
  -3.14751573e-03  2.95692775e-03  1.46104543e-03  6.51493090e-03
   4.66639416e-03  2.17151208e-03 -1.72292217e-01 -1.71328581e-01
  -1.70098885e-01 -1.64962980e-01 -1.67066805e-01 -1.75319402e-01
  -1.72832666e-01 -1.69558043e-01 -1.67746918e-01 -1.70235395e-01
   1.07648701e-03  1.60027621e-03 -1.32564933e-03  3.03782265e-04
  -1.35286280e-03 -1.12834508e-04  1.16567182e-03  6.00594343e-04
  -2.44194132e-03 -2.29453809e-04 -4.25020642e-02 -4.68335511e-02
  -4.21142883e-02 -4.33971569e-02 -5.13880206e-02 -4.39840573e-02
  -4.08359493e-02 -4.61165958e-02 -5.20044894e-02 -4.54025155e-02
   1.90349109e-06  6.81378742e-08  3.75097974e-09  3.37352923e-06
  -4.64429081e-08  5.16500655e-08  1.94957814e-07  7.86439405e-08
   6.50124624e-08  1.59656570e-07]]


In [99]:
print(coef_dict_k_deg["LS"])

[[-7.83934680e-04 -3.51271120e-03 -1.08350244e-04  1.68767505e-03
  -1.65652623e-03  3.88930442e-03 -5.76725384e-04  8.13588943e-03
   3.29762059e-03  1.31565910e-03 -1.72846782e-01 -1.71135688e-01
  -1.71325045e-01 -1.64174020e-01 -1.67072009e-01 -1.73155660e-01
  -1.72642422e-01 -1.69596028e-01 -1.70011114e-01 -1.69981504e-01
   1.00010598e-03  1.91877161e-03 -7.76019520e-04  9.82619800e-05
  -1.26701189e-03  9.32862132e-05  9.12826818e-04  2.84369051e-04
  -2.21947236e-03 -3.96866475e-04 -6.85977565e-02 -7.33238187e-02
  -6.81577886e-02 -7.16838144e-02 -7.89232957e-02 -7.19167743e-02
  -6.85763164e-02 -7.54581535e-02 -8.01178045e-02 -7.20225556e-02
   2.21388419e-06  8.11157262e-08  1.03148920e-08  3.85264809e-06
  -3.53221783e-08  5.75969522e-08  2.16768469e-07  4.72928529e-08
   1.74343371e-07  1.68518104e-07]]


In [100]:
print(coef_dict_k_deg["MAX"])

[[-1.19692917e-02 -2.17550985e-02  1.15400995e-02  2.99583626e-02
  -2.36230305e-02  1.58707680e-02  1.64125940e-02  2.97525767e-02
  -1.14751537e-02  5.29441594e-03 -1.32709169e-01 -1.38870314e-01
  -1.58262815e-01 -1.45684574e-01 -1.54873167e-01 -1.59789010e-01
  -1.85317934e-01 -1.71472442e-01 -1.29547812e-01 -1.25374663e-01
   3.44772945e-03  6.91514963e-03  5.09827693e-03  6.73418686e-03
  -1.92671838e-03 -7.15671549e-03  1.34661287e-02  9.66682911e-03
  -1.72573094e-02  4.73927959e-03 -8.29729727e-02 -7.35131985e-02
  -1.17240656e-01 -7.33659894e-02 -1.21722365e-01 -6.32829954e-02
  -3.04180730e-02 -8.65988867e-02 -1.18065292e-01 -9.59154786e-02
   3.75597371e-06  1.21342567e-07 -1.89589432e-06 -2.87864136e-06
  -2.77154639e-07 -3.55170903e-07  8.83881844e-07  5.45788549e-07
  -1.03794576e-06  8.94946885e-08]]


### Test 

In [101]:
#Create a dictionary and put respective matrices into it
test_params = {
    "W":W_test_spec,
    "step":None,
    "burn_in":None,
    "n_test":N_test,
    "dim":d
}
nbcores = multiprocessing.cpu_count()
trav = Pool(nbcores)
res = trav.starmap(Run_eval_test, [(i,degree,sampler,methods,inds_arr,Cur_pot,test_params,coef_dict_k_deg,params,f_type) for i in range (n_traj_test)])
trav.close()

In [102]:
methods_enh = ['Vanilla'] + methods
print(methods_enh)
ints_result = {key: [] for key in methods_enh}
vars_result = {key: [] for key in methods_enh}

['Vanilla', 'EVM', 'LS', 'MAX']


In [103]:
for i in range(len(res)):
    for j in range(len(methods_enh)):
        ints_result[methods_enh[j]].append(res[i][0][methods_enh[j]][0])
        vars_result[methods_enh[j]].append(res[i][1][methods_enh[j]][0])
for key in methods_enh:
    ints_result[key] = np.asarray(ints_result[key])
    vars_result[key] = np.asarray(vars_result[key])

### Results

In [104]:
print("Estimators")
for i in range(len(methods_enh)):
    print(methods_enh[i])
    print("mean: ",np.mean(ints_result[methods_enh[i]],axis=0))
    print("std: ",np.std(ints_result[methods_enh[i]],axis=0))
    print("max deviation: ",np.max(np.abs(ints_result[methods_enh[i]] - np.mean(ints_result[methods_enh[i]]))))

Estimators
Vanilla
mean:  [3.74229329]
std:  [0.01859722]
max deviation:  0.0608764114161362
EVM
mean:  [1.14981546]
std:  [2.30510797]
max deviation:  22.46425014966313
LS
mean:  [-0.28294848]
std:  [2.67970185]
max deviation:  26.136577810303223
MAX
mean:  [-1.14301184]
std:  [4.69576955]
max deviation:  44.380667065039866


In [105]:
print("Variances")
for i in range(len(methods_enh)):
    print(methods_enh[i])
    print(np.mean(vars_result[methods_enh[i]],axis=0))

Variances
Vanilla
[3.03368318]
EVM
[54067.79003951]
LS
[72857.93394204]
MAX
[242669.44687005]


In [ ]:
#save results
res_dict = {"int":ints_result,"var":vars_result}
np.save("Results/15_09/MC_Pareto_sum_cos_traj_1_d_10_beta_1e-1_train_5e2_test_1e4_deg_3.npy",res_dict)

### Plot results

In [ ]:
var_ind = 0
title = ""
labels = ['Vanilla', 'EVM', 'LS','MAX']

In [ ]:
# Box plot
data = [res_dict['int'][method][:,var_ind] for method in labels] 
boxplot_ind(data, title, labels)

In [ ]:
var_ind = 0
title = ""
labels = ['EVM','MAX']
data = [res_dict['int'][method][:,var_ind] for method in labels] 
boxplot_ind(data, title, labels)

In [ ]:
var_ind = 0 #Index to plot
title = ""
labels = ['ULA \nwith EVM','ULA\nwith ESVM']

In [ ]:
data = [results[:,0,4,var_ind]-1.25,results[:,0,2,var_ind]-1.25] 
boxplot_ind(data, title, labels)

In [ ]:
vars_vanilla = results[:,1,0,:]
vars_esvm_1st = results[:,1,1,:]
vars_esvm_2nd = results[:,1,2,:]
vars_evm_1st = results[:,1,3,:]
vars_evm_2nd = results[:,1,4,:]
print("average VRF for 1st order EVM:",np.mean(vars_vanilla)/np.mean(vars_evm_1st))
print("average VRF for 2nd order EVM:",np.mean(vars_vanilla)/np.mean(vars_evm_2nd))
print("average VRF for 1st order ESVM:",np.mean(vars_vanilla)/np.mean(vars_esvm_1st))
print("average VRF for 2nd order ESVM:",np.mean(vars_vanilla)/np.mean(vars_esvm_2nd))